<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/CLASSIFICATION1_multi_task_shkim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00


# AG NEWS DataSet Label 추가

In [ ]:
from datasets import load_dataset, ClassLabel, DatasetDict

agnews_dataset = load_dataset("fancyzhx/ag_news")
print(agnews_dataset['train'].features)

new_labels = agnews_dataset["train"].features["label"].names + ["StackOverFlow"]
new_class_label = ClassLabel(names=new_labels)

# 새로운 클래스 레이블을 기존 데이터셋에 적용하기 위해 데이터셋을 업데이트합니다.
updated_features = agnews_dataset["train"].features.copy()
updated_features["label"] = new_class_label

agnews_dataset = agnews_dataset.cast(updated_features)

# 결과 출력
print(agnews_dataset['train'].features)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}


Casting the dataset:   0%|          | 0/120000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech', 'StackOverFlow'], id=None)}


# TrainSet에 데이터 추가

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value

sof_dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/convert_train.csv',
    split='train'
)
# print(sof_dataset_train)

class_label = ClassLabel(num_classes=5, names=['World', 'Sports', 'Business', 'Sci/Tech', 'StackOverFlow'])

# 데이터셋에서 30000건 추출
sof_train_samples = sof_dataset_train.shuffle(seed=42).select(range(30000))
# 'Concat_Text' 필드를 'text'로, 그리고 'label' 값을 'StackOverFlow'로 설정
sof_train_samples = sof_train_samples.map(lambda example: {'text': example['Concat_Text'], 'label': class_label.str2int('StackOverFlow')}, remove_columns=['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y', 'Cleaned_tag ', 'Cleaned_tag', 'Array_Tag', 'Filtered_Array_Tag', 'Filtered_Tag', 'Concat_Text'])

# sof_train_samples의 features를 업데이트
sof_train_samples = sof_train_samples.cast_column('label', class_label)

# sof_train_samples와 agnews_train을 결합
combined_train_dataset = concatenate_datasets([agnews_dataset["train"], sof_train_samples])

# 결과를 확인
print(combined_train_dataset)

# 레이블의 데이터 건수 카운트
label_counts = combined_train_dataset.features['label'].num_classes
label_counter = {label: 0 for label in range(label_counts)}

for example in combined_train_dataset:
    label_counter[example['label']] += 1

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 150000
})
{0: 30000, 1: 30000, 2: 30000, 3: 30000, 4: 30000}


# TestSet에 데이터 추가

In [ ]:
sof_dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/convert_valid.csv',
    split='train'
)
sof_test_samples = sof_dataset_test.shuffle(seed=42).select(range(1900))
# 'Concat_Text' 필드를 'text'로, 그리고 'label' 값을 'StackOverFlow'로 설
sof_test_samples = sof_test_samples.map(lambda example: {'text': example['Concat_Text'], 'label': class_label.str2int('StackOverFlow')}, remove_columns=['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y', 'Cleaned_tag ', 'Cleaned_tag', 'Array_Tag', 'Filtered_Array_Tag', 'Filtered_Tag', 'Concat_Text'])

# sof_train_samples의 features를 업데이트
sof_test_samples = sof_test_samples.cast_column('label', class_label)

combined_test_dataset = concatenate_datasets([agnews_dataset["test"], sof_test_samples])

# 결과 확인
print(combined_test_dataset)

# 레이블의 데이터 건수 카운트
label_counts = combined_test_dataset.features['label'].num_classes
label_counter = {label: 0 for label in range(label_counts)}

for example in combined_test_dataset:
    label_counter[example['label']] += 1

print(label_counter)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1900 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 9500
})
{0: 1900, 1: 1900, 2: 1900, 3: 1900, 4: 1900}


In [ ]:
from transformers import AutoTokenizer

new_agnews_dataset= DatasetDict({
    "train": combined_train_dataset,
    "test": combined_test_dataset
})

# DatasetDict 출력
print(new_agnews_dataset)

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = new_agnews_dataset.map(tokenize_function, batched=True)



small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1500))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9500
    })
})


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

# **Multi task 클래스 정의 및 테스트**

In [ ]:
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
from torch import nn
import numpy as np
import evaluate
from peft import LoraConfig, TaskType
from peft import get_peft_model

# RoBERTa 모델 로드
config = RobertaConfig.from_pretrained('FacebookAI/roberta-large')
base_model = RobertaModel.from_pretrained('FacebookAI/roberta-large', config=config)

# 모델의 모든 파라미터를 동결
for param in base_model.parameters():
    param.requires_grad = False

# 두 개의 분류 헤드를 추가하기 위해 nn.Module을 상속받아 클래스를 정의
class RobertaTwoHeadsModel(nn.Module):
    def __init__(self, base_model, config_task1, num_labels_task1):
        super().__init__()
        self.config = config  # 모델 구성을 저장하는 config 속성 추가
        self.base_model = base_model
        # Task 1의 분류기
        self.classifier_task1 = nn.Linear(config.hidden_size, num_labels_task1)
        # Task 2의 분류기
        self.classifier_task2 = nn.Linear(config.hidden_size, num_labels_task2)

        # 첫 번째 분류 헤드의 파라미터에 대해서만 동결을 해제
        for param in self.classifier_task1.parameters():
            param.requires_grad = True

        # 두 번째 분류 헤드의 파라미터에 대해서만 동결을 해제
        for param in self.classifier_task2.parameters():
            param.requires_grad = True

    def forward(self, input_ids=None, attention_mask=None, labels=None, task=None, **kwargs):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        cls_token_state = sequence_output[:, 0]

        loss = None
        loss_fct = nn.CrossEntropyLoss()
        if task == 1:
            logits = self.classifier_task1(cls_token_state)
            if labels is not None:
                loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))
        elif task == 2:
            logits = self.classifier_task2(cls_token_state)
            if labels is not None:
                loss = loss_fct(logits.view(-1, self.num_labels_task2), labels.view(-1))
        else:
            raise ValueError("Task not defined")

        return SequenceClassifierOutput(loss=loss, logits=logits)

# Task 1의 레이블 수와 Config 설정
num_labels_task1 = 5
config_task1 = config
config_task1.num_labels = num_labels_task1

# Task 2의 레이블 수
num_labels_task2 = 3

# 사용자 정의 모델 인스턴스화
model = RobertaTwoHeadsModel(base_model, config_task1, num_labels_task1)

# 모델 구조를 출력
# print(model)

# 모델 구조에 따라 target_modules 경로를 업데이트
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=[
        f"base_model.encoder.layer.{i}.attention.self.query" for i in range(config.num_hidden_layers)] +
        [f"base_model.encoder.layer.{i}.attention.self.key" for i in range(config.num_hidden_layers)] +
        [f"base_model.encoder.layer.{i}.attention.self.value" for i in range(config.num_hidden_layers)] +
        [f"base_model.encoder.layer.{i}.attention.output.dense" for i in range(config.num_hidden_layers)] +
        [f"base_model.encoder.layer.{i}.intermediate.dense" for i in range(config.num_hidden_layers)] +
        [f"base_model.encoder.layer.{i}.output.dense" for i in range(config.num_hidden_layers)]
)

model = get_peft_model(model, lora_config)

# 메트릭을 계산하기 위한 함수 정의
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
    }
    return metrics

# 사용자 정의 MultiTaskTrainer 클래스
class MultiTaskTrainer(Trainer):
    def __init__(self, *args, num_labels=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_labels = num_labels  # 현재 태스크의 레이블 수를 저장

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        task = inputs.pop("task", 1)  # 'task'가 딕셔너리에 없으면 기본값 1을 사용

        outputs = model(**inputs, labels=labels, task=task)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=10
)


# Task 1을 위한 MultiTaskTrainer 초기화 및 훈련
trainer = MultiTaskTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,  # Task 1의 훈련 데이터셋
    eval_dataset=small_eval_dataset,    # Task 1의 평가 데이터셋
    compute_metrics=compute_metrics,
    num_labels=num_labels_task1,
)

# Task 1 훈련
trainer.train()


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.364482,0.895333,0.905135,0.895454,0.896879
2,No log,0.261159,0.927333,0.929157,0.927981,0.928377
3,0.601800,0.305870,0.926667,0.927663,0.928023,0.927410
4,0.601800,0.309438,0.932000,0.932574,0.932616,0.932410
5,0.601800,0.308289,0.925333,0.925758,0.926594,0.925815
6,0.210500,0.316142,0.932000,0.932652,0.932971,0.932670
7,0.210500,0.349750,0.930000,0.930213,0.931021,0.930461
8,0.142900,0.335877,0.931333,0.931950,0.932401,0.932045
9,0.142900,0.340532,0.933333,0.933910,0.933978,0.933901
10,0.142900,0.348527,0.928667,0.928969,0.929573,0.929224


TrainOutput(global_step=1880, training_loss=0.2754951071231923, metrics={'train_runtime': 1396.9759, 'train_samples_per_second': 10.737, 'train_steps_per_second': 1.346, 'total_flos': 0.0, 'train_loss': 0.2754951071231923, 'epoch': 10.0})

# **추론메소드 정의**


In [ ]:
# 모델과 데이터를 같은 디바이스로 이동시키는 코드 추가
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # 모델을 적절한 디바이스로 이동

def inference(model, tokenizer, text, task=1):
    model.eval()  # 모델을 평가 모드로 설정

    # 텍스트를 토크나이즈하고 텐서로 변환한 후 적절한 디바이스로 이동
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():  # 그래디언트 계산을 비활성화
        outputs = model(**inputs, task=task)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    top_pred = probabilities.argmax(dim=-1)

    # 예측 결과를 CPU로 이동시킨 후 numpy 배열로 변환
    return probabilities.detach().cpu().numpy(), top_pred.detach().cpu().numpy()

# **Inference**

In [ ]:
label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech",
    4: "StackOverFlow"
}

# 예제 텍스트
sample_texts = [
    "US trade deficit swells in June The US trade deficit has exploded 19 to a record \$55.8bn as oil costs drove imports higher, according to a latest figures.",
    "Indians Beat Twins 7-1, Nearing AL Lead (AP) AP - The Cleveland Indians pulled within one game of the AL Central lead, scoring four runs in the first inning and beating the Minnesota Twins 7-1 Saturday night behind home runs by Travis Hafner and Victor Martinez.",
    "Pilgrims Crowd Field for Mass With Pope LOURDES, France - A frail Pope John Paul II celebrated an open-air Mass on Sunday as several hundred thousand pilgrims, many in wheelchairs, crowded onto a field near a French shrine to the Virgin Mary that is associated with miraculous cures of the sick. The Mass was a highlight of the Pope's two-day visit to Lourdes, a town in the Pyrenees where Roman Catholic tradition says St...",
    "T. Rex Had Teen Growth Spurt, Scientists Say (Reuters) Reuters - Tyrannosaurus Rex grew incredibly fast\during a teenaged growth spurt that saw the dinosaur expand its\bulk by six times, but the fearsome beasts \"lived fast and died\young,\" researchers said on Wednesday.",
    "working of compareTo() method of Comparable interface            I have one Employee class and the requirement is to sort the objects using comparable interface. The output with this code is :The difference of this id and other id is..** 6  other id**1The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**1The difference of this id and other id is..** 11  other id**3The difference of this id and other id is..** 11  other id**6",
]

# Task 1에 대한 추론을 각 텍스트에 대해 수행하고 결과를 출력
for i, text in enumerate(sample_texts):
    probabilities, top_pred = inference(model, tokenizer, text, task=1)
    predicted_label = label_map[top_pred[0]]
    predicted_probability = probabilities[0][top_pred[0]]

    print(f"Text {i+1} Predicted label: {predicted_label}, Score: {predicted_probability:.4f}")

Text 1 Predicted label: Business, Score: 0.9989
Text 2 Predicted label: Sports, Score: 0.9973
Text 3 Predicted label: World, Score: 0.9998
Text 4 Predicted label: Sci/Tech, Score: 0.9978
Text 5 Predicted label: StackOverFlow, Score: 0.9998
